# Topic Models for SN Users
Count how many threads special needs users participate it. It is almost the total number of threads. Thus, a model on all threads SN users participate in is not useful for the use case of evaluating what SN parents talk about. Use a model trained on SN threads instead.

In [1]:
import pandas as pd
from pathlib import Path
from scraping import create_connection

In [2]:
p = Path.cwd()
path_parent = p.parents[1]

In [3]:
# database
path_db = str(path_parent / "database" / "netmums-merged.db")
path_clean_data = path_parent / "clean_data" / "netmums"

In [16]:
sql = """
WITH snusers AS (
    SELECT COUNT(*) as n_posts, p.user_url AS user_url
    FROM posts AS p
    LEFT JOIN threads AS t
    ON t.id=p.thread_id
    LEFT JOIN subforums AS s
    ON s.id=t.subforum_id
    WHERE s.forum_id=24
    GROUP BY user_url
), snuserthreads AS (
    SELECT DISTINCT p.thread_id AS thread_id
    FROM posts AS p
    INNER JOIN snusers
    ON p.user_url=snusers.user_url
    WHERE snusers.n_posts > 50
)
SELECT
    p.thread_id AS thread_id,
    text.post_id AS post_id,
    text.text_clean AS text_clean
FROM text
LEFT JOIN posts AS p
ON text.post_id = p.id
INNER JOIN snuserthreads AS sn
ON p.thread_id=sn.thread_id
"""

In [17]:
conn = create_connection(path_db)
df = pd.read_sql_query(sql, conn)

In [18]:
conn.close()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002935 entries, 0 to 15002934
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   thread_id   int64 
 1   post_id     int64 
 2   text_clean  object
dtypes: int64(2), object(1)
memory usage: 343.4+ MB


In [4]:
sql = """
SELECT COUNT(*)
FROM text
"""

In [7]:
df

,COUNT(*)
0,15888144
